In [1]:
pip install xlsxwriter openpyxl

     ---------------------------------------- 0.0/159.9 kB ? eta -:--:--
     -- ------------------------------------- 10.2/159.9 kB ? eta -:--:--
     ------- ----------------------------- 30.7/159.9 kB 330.3 kB/s eta 0:00:01
     ------- ----------------------------- 30.7/159.9 kB 330.3 kB/s eta 0:00:01
     ------- ----------------------------- 30.7/159.9 kB 330.3 kB/s eta 0:00:01
     --------- --------------------------- 41.0/159.9 kB 131.3 kB/s eta 0:00:01
     -------------- ---------------------- 61.4/159.9 kB 193.2 kB/s eta 0:00:01
     ------------------------- ---------- 112.6/159.9 kB 328.2 kB/s eta 0:00:01
     --------------------------- -------- 122.9/159.9 kB 313.8 kB/s eta 0:00:01
     ------------------------------------ 159.9/159.9 kB 354.7 kB/s eta 0:00:00
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from datetime import datetime,timedelta
import xlsxwriter
import openpyxl
from openpyxl.styles import PatternFill

In [3]:

classes_taken_dates = ["06/08/2024", "13/08/2024", "20/08/2024", "27/08/2024",
                       "03/09/2024", "17/09/2024", "01/10/2024"]
classes_missed_dates = ["10/09/2024"]
exams_dates = ["24/09/2024"]
all_dates = sorted(classes_taken_dates + classes_missed_dates + exams_dates, key=lambda x: datetime.strptime(x, '%d/%m/%Y'))  
class_timing = "18:00 - 20:00"

In [4]:
# 6-8pm
def is_present(attendance_time, lecture_start, lecture_end):
    attendance_time = datetime.strptime(attendance_time, '%d/%m/%Y %H:%M:%S') 
    return lecture_start <= attendance_time <= lecture_end

In [6]:
with open(r'C:\Users\draka\Downloads\temp\stud_list.txt', 'r') as file:
    students = [line.split()[0] for line in file.read().splitlines()]  

attendance_df = pd.read_csv(r'C:\Users\draka\Downloads\temp\input_attendance.csv', names=['Timestamp', 'Roll'], header=0)


In [7]:
attendance_df['Roll'] = attendance_df['Roll'].fillna('')  

attendance_df['Roll'] = attendance_df['Roll'].apply(lambda x: x.split()[0] if x else '')  

output_excel = 'output_excel.xlsx'
writer = pd.ExcelWriter(output_excel, engine='openpyxl')

attendance_summary = []
proxy_summary = []

In [8]:
for date in all_dates:
    lecture_start = datetime.strptime(f'{date} 18:00', '%d/%m/%Y %H:%M')
    lecture_end = lecture_start + timedelta(hours=2)  

   
    daily_attendance = {}
    daily_proxy = {}

    for student in students:
        
        student_records = attendance_df[(attendance_df['Roll'] == student) &
                                        (attendance_df['Timestamp'].str.startswith(date))]

        
        present_count = student_records['Timestamp'].apply(lambda x: is_present(x, lecture_start, lecture_end)).sum()

        
        attendance_status = present_count  

        # for proxyy
        proxy_count = present_count - 2 if present_count > 2 else 0

        daily_attendance[student] = attendance_status
        daily_proxy[student] = proxy_count

    attendance_summary.append(daily_attendance)
    proxy_summary.append(daily_proxy)

In [9]:
attendance_df = pd.DataFrame(attendance_summary, index=all_dates)
proxy_df = pd.DataFrame(proxy_summary, index=all_dates)

attendance_df = attendance_df.T
proxy_df = proxy_df.T

attendance_df['Total Count'] = attendance_df.count(axis=1) 
attendance_df['Total Attendance Marked'] = attendance_df.iloc[:, :-1].sum(axis=1)  
attendance_df['Total Attendance Allowed'] = len(classes_taken_dates) * 2  
attendance_df['Total Proxy'] = proxy_df.sum(axis=1)  

In [10]:
attendance_df.to_excel(writer, sheet_name='Attendance')

workbook = writer.book
sheet = workbook['Attendance']

red_fill = PatternFill(start_color='FF9999', end_color='FF9999', fill_type='solid')
yellow_fill = PatternFill(start_color='FFFF99', end_color='FFFF99', fill_type='solid')
green_fill = PatternFill(start_color='99FF99', end_color='99FF99', fill_type='solid')

for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=2, max_col=sheet.max_column - 4):
    for cell in row:
        if cell.value == 0:
            cell.fill = red_fill
        elif cell.value == 1:
            cell.fill = yellow_fill
        elif cell.value >= 2:
            cell.fill = green_fill

writer.close()